In [ ]:
# %% [markdown]
# # Ottimizzazione SRK/T per Faco-DMEK - CON DATI REALI
# 
# Questo notebook usa SOLO i dati reali dal file FacoDMEK.xlsx

# %% [code]
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.optimize import minimize
from sklearn.model_selection import KFold
import warnings
warnings.filterwarnings('ignore')

print("📂 Caricamento dati REALI da FacoDMEK.xlsx...")

# %% [code]
# CARICA I DATI REALI
df = pd.read_excel('FacoDMEK.xlsx', sheet_name='Data')

# Pulisci i dati - rimuovi righe con valori mancanti nelle colonne essenziali
required_cols = ['Bio-AL', 'Bio-Ks', 'Bio-Kf', 'IOL Power', 'A-Constant', 'PostOP Spherical Equivalent']
df_clean = df.dropna(subset=required_cols).copy()

# Calcola K medio
df_clean['K_mean'] = (df_clean['Bio-Ks'] + df_clean['Bio-Kf']) / 2

print(f"✅ Dataset caricato: {len(df_clean)} pazienti con dati completi")
print(f"📊 Colonne disponibili: {df.shape[1]}")
print(f"📏 Range AL: {df_clean['Bio-AL'].min():.1f} - {df_clean['Bio-AL'].max():.1f} mm")
print(f"👁️ Range K medio: {df_clean['K_mean'].min():.1f} - {df_clean['K_mean'].max():.1f} D")

# Mostra prime righe
print("\nPrime 5 righe dei dati:")
df_clean[['ID', 'Bio-AL', 'K_mean', 'IOL Power', 'A-Constant', 'PostOP Spherical Equivalent']].head()

# %% [markdown]
# ## Implementazione Formula SRK/T Standard

# %% [code]
def calculate_srkt(AL, K, IOL_power, A_const, n_kerat=337.5, nc=1.333):
    """
    Calcola la refrazione prevista con la formula SRK/T
    
    Parametri:
    - AL: lunghezza assiale
    - K: cheratometria media
    - IOL_power: potere della IOL impiantata
    - A_const: A-constant della IOL
    - n_kerat: indice cheratometrico (default 337.5)
    - nc: indice di rifrazione corneale (default 1.333)
    """
    # Costanti fisse
    na = 1.336  # indice rifrazione acqueo/vitreo
    V = 12      # distanza vertice
    ncm1 = nc - 1
    
    # Step 1: Raggio corneale
    r = n_kerat / K
    
    # Step 2: LCOR (Corrected Axial Length)
    if AL <= 24.2:
        LCOR = AL
    else:
        LCOR = -3.446 + 1.716 * AL - 0.0237 * AL**2
    
    # Step 3: Larghezza corneale (Cw)
    Cw = -5.41 + 0.58412 * LCOR + 0.098 * K
    
    # Step 4: Altezza corneale (H)
    x = r**2 - (Cw**2) / 4
    if x < 0:
        H = 0
    else:
        H = r - np.sqrt(x)
    
    # Step 5-6: ACDest
    ACDconst = 0.62467 * A_const - 68.747
    offset = ACDconst - 3.336
    ACDest = H + offset
    
    # Step 7-8: RETHICK e LOPT
    RETHICK = 0.65696 - 0.02029 * AL
    LOPT = AL + RETHICK
    
    # Step 9: Calcolo refrazione prevista
    num = (1000 * na * (na * r - ncm1 * LOPT) - 
           IOL_power * (LOPT - ACDest) * (na * r - ncm1 * ACDest))
    
    den = (na * (V * (na * r - ncm1 * LOPT) + LOPT * r) - 
           0.001 * IOL_power * (LOPT - ACDest) * 
           (V * (na * r - ncm1 * ACDest) + ACDest * r))
    
    return num / den

# %% [markdown]
# ## Calcolo Baseline con Parametri Standard

# %% [code]
# Calcola predizioni con parametri STANDARD per ogni paziente
predictions_baseline = []
for idx, row in df_clean.iterrows():
    pred = calculate_srkt(
        AL=row['Bio-AL'],
        K=row['K_mean'],
        IOL_power=row['IOL Power'],
        A_const=row['A-Constant'],
        n_kerat=337.5,  # standard
        nc=1.333        # standard
    )
    predictions_baseline.append(pred)

df_clean['SE_previsto_baseline'] = predictions_baseline
df_clean['Errore_baseline'] = df_clean['PostOP Spherical Equivalent'] - df_clean['SE_previsto_baseline']
df_clean['Errore_assoluto_baseline'] = np.abs(df_clean['Errore_baseline'])

# Calcola metriche REALI
MAE_baseline = df_clean['Errore_assoluto_baseline'].mean()
RMSE_baseline = np.sqrt((df_clean['Errore_baseline']**2).mean())
ME_baseline = df_clean['Errore_baseline'].mean()

# Percentuali
within_05_baseline = (df_clean['Errore_assoluto_baseline'] <= 0.5).sum() / len(df_clean) * 100
within_10_baseline = (df_clean['Errore_assoluto_baseline'] <= 1.0).sum() / len(df_clean) * 100
within_20_baseline = (df_clean['Errore_assoluto_baseline'] <= 2.0).sum() / len(df_clean) * 100

print("=== RISULTATI BASELINE (DATI REALI) ===")
print(f"MAE:  {MAE_baseline:.3f} D")
print(f"RMSE: {RMSE_baseline:.3f} D")
print(f"ME (bias): {ME_baseline:.3f} D")
print(f"\nDistribuzione errori:")
print(f"Entro ±0.5 D: {within_05_baseline:.1f}% ({int(within_05_baseline * len(df_clean) / 100)} pazienti)")
print(f"Entro ±1.0 D: {within_10_baseline:.1f}% ({int(within_10_baseline * len(df_clean) / 100)} pazienti)")
print(f"Entro ±2.0 D: {within_20_baseline:.1f}% ({int(within_20_baseline * len(df_clean) / 100)} pazienti)")

# Direzione errori
iper = (df_clean['Errore_baseline'] > 0).sum()
miop = (df_clean['Errore_baseline'] < 0).sum()
print(f"\nDirezione errori:")
print(f"Tendenza ipermetropica (SE reale > previsto): {iper} ({iper/len(df_clean)*100:.1f}%)")
print(f"Tendenza miopica (SE reale < previsto): {miop} ({miop/len(df_clean)*100:.1f}%)")

# %% [code]
# Visualizzazione baseline
fig, axes = plt.subplots(2, 2, figsize=(12, 10))

# 1. Istogramma errori
ax1 = axes[0, 0]
ax1.hist(df_clean['Errore_baseline'], bins=20, alpha=0.7, color='blue', edgecolor='black')
ax1.axvline(0, color='red', linestyle='--', label='Zero error')
ax1.axvline(ME_baseline, color='orange', linestyle='--', label=f'Bias = {ME_baseline:.2f}D')
ax1.set_xlabel('Errore Refrattivo (D)')
ax1.set_ylabel('Frequenza')
ax1.set_title('Distribuzione Errori - Baseline')
ax1.legend()

# 2. Previsto vs Reale
ax2 = axes[0, 1]
ax2.scatter(df_clean['SE_previsto_baseline'], df_clean['PostOP Spherical Equivalent'], alpha=0.6)
ax2.plot([-4, 4], [-4, 4], 'r--', label='Perfetta predizione')
ax2.set_xlabel('SE Previsto (D)')
ax2.set_ylabel('SE Reale (D)')
ax2.set_title('Previsto vs Reale')
ax2.legend()
ax2.set_xlim(-4, 4)
ax2.set_ylim(-4, 4)

# 3. Errore vs AL
ax3 = axes[1, 0]
ax3.scatter(df_clean['Bio-AL'], df_clean['Errore_baseline'], alpha=0.6)
ax3.axhline(0, color='red', linestyle='--')
ax3.set_xlabel('Lunghezza Assiale (mm)')
ax3.set_ylabel('Errore (D)')
ax3.set_title('Errore vs AL')

# 4. Errore vs K
ax4 = axes[1, 1]
ax4.scatter(df_clean['K_mean'], df_clean['Errore_baseline'], alpha=0.6)
ax4.axhline(0, color='red', linestyle='--')
ax4.set_xlabel('K medio (D)')
ax4.set_ylabel('Errore (D)')
ax4.set_title('Errore vs K')

plt.tight_layout()
plt.show()

# %% [markdown]
# ## Analisi per Sottogruppi

# %% [code]
print("=== ANALISI ERRORI PER SOTTOGRUPPI ===\n")

# Per AL
print("1. PER LUNGHEZZA ASSIALE:")
for nome, mask in [
    ('Corti (<22 mm)', df_clean['Bio-AL'] < 22),
    ('Normali (22-24.5 mm)', (df_clean['Bio-AL'] >= 22) & (df_clean['Bio-AL'] <= 24.5)),
    ('Lunghi (>24.5 mm)', df_clean['Bio-AL'] > 24.5)
]:
    subset = df_clean[mask]
    if len(subset) > 0:
        mae = subset['Errore_assoluto_baseline'].mean()
        me = subset['Errore_baseline'].mean()
        entro05 = (subset['Errore_assoluto_baseline'] <= 0.5).sum() / len(subset) * 100
        print(f"\n{nome} (n={len(subset)}):")
        print(f"  MAE: {mae:.3f} D")
        print(f"  ME (bias): {me:.3f} D")
        print(f"  Entro ±0.5 D: {entro05:.1f}%")

# Per K
print("\n2. PER CHERATOMETRIA:")
for nome, mask in [
    ('Piatte (<42 D)', df_clean['K_mean'] < 42),
    ('Normali (42-44 D)', (df_clean['K_mean'] >= 42) & (df_clean['K_mean'] <= 44)),
    ('Curve (>44 D)', df_clean['K_mean'] > 44)
]:
    subset = df_clean[mask]
    if len(subset) > 0:
        mae = subset['Errore_assoluto_baseline'].mean()
        me = subset['Errore_baseline'].mean()
        print(f"\n{nome} (n={len(subset)}):")
        print(f"  MAE: {mae:.3f} D")
        print(f"  ME (bias): {me:.3f} D")

# %% [markdown]
# ## Ottimizzazione Grid Search

# %% [code]
def objective_mae(params):
    """Funzione obiettivo: calcola MAE con dati parametri"""
    n_kerat, nc = params
    predictions = []
    for idx, row in df_clean.iterrows():
        pred = calculate_srkt(
            AL=row['Bio-AL'],
            K=row['K_mean'],
            IOL_power=row['IOL Power'],
            A_const=row['A-Constant'],
            n_kerat=n_kerat,
            nc=nc
        )
        predictions.append(pred)
    
    errors = np.abs(df_clean['PostOP Spherical Equivalent'] - np.array(predictions))
    return errors.mean()

# Grid search
print("🔍 Esecuzione Grid Search...")
n_kerat_range = np.arange(335, 340.5, 0.5)
nc_range = np.arange(1.330, 1.336, 0.001)

results_grid = []
best_mae = float('inf')
best_params = None

for n_kerat in n_kerat_range:
    for nc in nc_range:
        mae = objective_mae([n_kerat, nc])
        results_grid.append({
            'n_kerat': n_kerat,
            'nc': nc,
            'MAE': mae
        })
        if mae < best_mae:
            best_mae = mae
            best_params = (n_kerat, nc)
            print(f"  Nuovo ottimo: n_kerat={n_kerat:.1f}, nc={nc:.4f}, MAE={mae:.3f}")

print(f"\n✅ Grid Search completata!")
print(f"Parametri ottimali: n_kerat={best_params[0]:.1f}, nc={best_params[1]:.4f}")
print(f"MAE ottimale: {best_mae:.3f} D")
print(f"Miglioramento: {(MAE_baseline - best_mae)/MAE_baseline*100:.1f}%")

# %% [code]
# Heatmap risultati
results_df = pd.DataFrame(results_grid)
pivot_mae = results_df.pivot(index='nc', columns='n_kerat', values='MAE')

plt.figure(figsize=(10, 8))
sns.heatmap(pivot_mae, annot=True, fmt='.3f', cmap='RdYlBu_r', 
            cbar_kws={'label': 'MAE (D)'})
plt.title('Grid Search: MAE per combinazioni di parametri')
plt.xlabel('n_kerat')
plt.ylabel('nc')
plt.tight_layout()
plt.show()

# %% [markdown]
# ## Ottimizzazione con Scipy

# %% [code]
# Ottimizzazione più precisa con scipy
from scipy.optimize import minimize

print("🎯 Ottimizzazione con scipy.minimize...")

# Punto di partenza
x0 = [337.5, 1.333]

# Bounds
bounds = [(335, 340), (1.330, 1.336)]

# Ottimizza
result = minimize(objective_mae, x0, method='L-BFGS-B', bounds=bounds)

n_kerat_opt = result.x[0]
nc_opt = result.x[1]
mae_opt = result.fun

print(f"\nParametri ottimali:")
print(f"n_kerat = {n_kerat_opt:.2f}")
print(f"nc = {nc_opt:.4f}")
print(f"MAE = {mae_opt:.3f} D")
print(f"Convergenza: {result.success}")
print(f"Iterazioni: {result.nit}")

# %% [markdown]
# ## Validazione con Cross-Validation

# %% [code]
print("📊 5-fold Cross Validation...")

kf = KFold(n_splits=5, shuffle=True, random_state=42)
cv_results = []

for fold, (train_idx, test_idx) in enumerate(kf.split(df_clean)):
    df_train = df_clean.iloc[train_idx]
    df_test = df_clean.iloc[test_idx]
    
    # Definisci funzione obiettivo per training set
    def objective_train(params):
        n_kerat, nc = params
        predictions = []
        for idx, row in df_train.iterrows():
            pred = calculate_srkt(row['Bio-AL'], row['K_mean'], 
                                row['IOL Power'], row['A-Constant'],
                                n_kerat, nc)
            predictions.append(pred)
        errors = np.abs(df_train['PostOP Spherical Equivalent'] - np.array(predictions))
        return errors.mean()
    
    # Ottimizza su training
    res = minimize(objective_train, [337.5, 1.333], 
                  method='L-BFGS-B', bounds=[(335, 340), (1.330, 1.336)])
    
    # Valuta su test
    predictions_test = []
    for idx, row in df_test.iterrows():
        pred = calculate_srkt(row['Bio-AL'], row['K_mean'], 
                            row['IOL Power'], row['A-Constant'],
                            res.x[0], res.x[1])
        predictions_test.append(pred)
    
    mae_test = np.abs(df_test['PostOP Spherical Equivalent'] - np.array(predictions_test)).mean()
    
    cv_results.append({
        'fold': fold + 1,
        'n_kerat': res.x[0],
        'nc': res.x[1],
        'mae_train': res.fun,
        'mae_test': mae_test
    })
    
    print(f"Fold {fold+1}: Train MAE={res.fun:.3f}, Test MAE={mae_test:.3f}")

cv_df = pd.DataFrame(cv_results)
print(f"\n📈 Risultati Cross-Validation:")
print(f"MAE medio test: {cv_df['mae_test'].mean():.3f} ± {cv_df['mae_test'].std():.3f}")
print(f"n_kerat medio: {cv_df['n_kerat'].mean():.2f} ± {cv_df['n_kerat'].std():.3f}")
print(f"nc medio: {cv_df['nc'].mean():.4f} ± {cv_df['nc'].std():.4f}")

# %% [markdown]
# ## Confronto Finale

# %% [code]
# Calcola predizioni con parametri ottimizzati
predictions_opt = []
for idx, row in df_clean.iterrows():
    pred = calculate_srkt(
        AL=row['Bio-AL'],
        K=row['K_mean'],
        IOL_power=row['IOL Power'],
        A_const=row['A-Constant'],
        n_kerat=n_kerat_opt,
        nc=nc_opt
    )
    predictions_opt.append(pred)

df_clean['SE_previsto_opt'] = predictions_opt
df_clean['Errore_opt'] = df_clean['PostOP Spherical Equivalent'] - df_clean['SE_previsto_opt']
df_clean['Errore_assoluto_opt'] = np.abs(df_clean['Errore_opt'])

# Metriche ottimizzate
MAE_opt = df_clean['Errore_assoluto_opt'].mean()
RMSE_opt = np.sqrt((df_clean['Errore_opt']**2).mean())
ME_opt = df_clean['Errore_opt'].mean()
within_05_opt = (df_clean['Errore_assoluto_opt'] <= 0.5).sum() / len(df_clean) * 100
within_10_opt = (df_clean['Errore_assoluto_opt'] <= 1.0).sum() / len(df_clean) * 100

print("=== CONFRONTO RISULTATI ===")
print("\nBASELINE (n_kerat=337.5, nc=1.333):")
print(f"  MAE: {MAE_baseline:.3f} D")
print(f"  RMSE: {RMSE_baseline:.3f} D")
print(f"  Bias: {ME_baseline:.3f} D")
print(f"  Entro ±0.5D: {within_05_baseline:.1f}%")
print(f"  Entro ±1.0D: {within_10_baseline:.1f}%")

print(f"\nOTTIMIZZATO (n_kerat={n_kerat_opt:.1f}, nc={nc_opt:.4f}):")
print(f"  MAE: {MAE_opt:.3f} D")
print(f"  RMSE: {RMSE_opt:.3f} D")
print(f"  Bias: {ME_opt:.3f} D")
print(f"  Entro ±0.5D: {within_05_opt:.1f}%")
print(f"  Entro ±1.0D: {within_10_opt:.1f}%")

print(f"\nMIGLIORAMENTO:")
print(f"  Riduzione MAE: {(MAE_baseline - MAE_opt)/MAE_baseline*100:.1f}%")
print(f"  Riduzione RMSE: {(RMSE_baseline - RMSE_opt)/RMSE_baseline*100:.1f}%")
print(f"  Riduzione bias: {abs(ME_baseline) - abs(ME_opt):.3f} D")

# %% [code]
# Visualizzazione confronto
fig, axes = plt.subplots(1, 2, figsize=(12, 5))

# Confronto distribuzioni errori
ax1 = axes[0]
ax1.hist(df_clean['Errore_baseline'], bins=20, alpha=0.5, label='Baseline', color='blue')
ax1.hist(df_clean['Errore_opt'], bins=20, alpha=0.5, label='Ottimizzato', color='green')
ax1.axvline(0, color='red', linestyle='--')
ax1.set_xlabel('Errore (D)')
ax1.set_ylabel('Frequenza')
ax1.set_title('Confronto Distribuzioni Errori')
ax1.legend()

# Box plot per sottogruppi
ax2 = axes[1]
data_plot = []
labels_plot = []
for nome, mask in [
    ('AL<22', df_clean['Bio-AL'] < 22),
    ('AL 22-24.5', (df_clean['Bio-AL'] >= 22) & (df_clean['Bio-AL'] <= 24.5)),
    ('AL>24.5', df_clean['Bio-AL'] > 24.5)
]:
    subset = df_clean[mask]
    if len(subset) > 5:
        data_plot.extend([
            subset['Errore_assoluto_baseline'].values,
            subset['Errore_assoluto_opt'].values
        ])
        labels_plot.extend([f'{nome}\nBase', f'{nome}\nOpt'])

ax2.boxplot(data_plot, labels=labels_plot)
ax2.set_ylabel('Errore Assoluto (D)')
ax2.set_title('Errori per Sottogruppi AL')
ax2.axhline(0.5, color='red', linestyle='--', alpha=0.5, label='±0.5D')

plt.tight_layout()
plt.show()

# %% [code]
# Salva risultati
output_summary = {
    'n_pazienti': len(df_clean),
    'baseline': {
        'n_kerat': 337.5,
        'nc': 1.333,
        'MAE': round(MAE_baseline, 3),
        'RMSE': round(RMSE_baseline, 3),
        'bias': round(ME_baseline, 3),
        'within_05D': round(within_05_baseline, 1),
        'within_10D': round(within_10_baseline, 1)
    },
    'ottimizzato': {
        'n_kerat': round(n_kerat_opt, 2),
        'nc': round(nc_opt, 4),
        'MAE': round(MAE_opt, 3),
        'RMSE': round(RMSE_opt, 3),
        'bias': round(ME_opt, 3),
        'within_05D': round(within_05_opt, 1),
        'within_10D': round(within_10_opt, 1)
    },
    'miglioramento_percentuale': {
        'MAE': round((MAE_baseline - MAE_opt)/MAE_baseline*100, 1),
        'RMSE': round((RMSE_baseline - RMSE_opt)/RMSE_baseline*100, 1)
    }
}

# Salva anche i dettagli per paziente
df_results = df_clean[['ID', 'Bio-AL', 'K_mean', 'IOL Power', 'A-Constant', 
                       'PostOP Spherical Equivalent', 'SE_previsto_baseline', 
                       'Errore_baseline', 'SE_previsto_opt', 'Errore_opt']].copy()

print("\n📁 Dati pronti per il salvataggio:")
print("- output_summary: dizionario con tutti i risultati")
print("- df_results: DataFrame con predizioni per ogni paziente")

# Se vuoi salvare:
# import json
# with open('srkt_optimization_results.json', 'w') as f:
#     json.dump(output_summary, f, indent=4)
# df_results.to_excel('srkt_predictions_comparison.xlsx', index=False)

print("\n✅ Analisi completata!")